* BITAmin7 github: https://github.com/BITAmin-bigdata/Bitamin7

# 웹크롤링 with DataFrame, Requests

* 목표: 네이버금융에서 삼성전자의 재무제표 크롤링
* 사이트: 네이버금융 삼성전자 https://finance.naver.com/
* 수집항목: 삼성전자 재무제표
* cf. https://engkimbs.tistory.com/625?category=762758

In [30]:
# Pandas와 Requests 불러오기
import pandas as pd
import requests

# url: 네이버금융에서 '삼성전자'를 검색한 웹페이지 주소
url = "https://finance.naver.com/item/main.nhn?code=005930"
# get: 서버에 자원 요청 
samsung_electronic = requests.get(url)
# text: 받아온 응답에서 우리가 원하는 html이 저장된 위치로 접근
html = samsung_electronic.text

# DataFrame의 read_html을 이용해서 재무제표 크롤링
financial_stmt = pd.read_html(samsung_electronic.text)[3]

# For DataFrame
financial_stmt.set_index(('주요재무정보', '주요재무정보', '주요재무정보'), inplace=True)
financial_stmt.index.rename('주요재무정보', inplace=True)
financial_stmt.columns = financial_stmt.columns.droplevel(2)
financial_stmt

최근 연간 실적                                       최근 분기 실적  \
              2018.12     2019.12     2020.12  2021.12(E)    2020.03   
주요재무정보                                                                 
매출액        2437714.00  2304009.00  2368070.00  2677586.00  553252.00   
영업이익        588867.00   277685.00   359939.00   499088.00   64473.00   
당기순이익       443449.00   217389.00   264078.00   376685.00   48849.00   
영업이익률           24.16       12.05       15.20       18.64      11.65   
순이익률            18.19        9.44       11.15       14.07       8.83   
ROE(지배주주)       19.63        8.69        9.98       13.52       8.45   
부채비율            36.97       34.12       37.07         NaN      34.19   
당좌비율           204.12      233.57      214.82         NaN     237.80   
유보율          27531.92    28856.02    30692.79         NaN   29134.12   
EPS(원)        6024.00     3166.00     3841.00     5512.00     720.00   
PER(배)           6.42       17.63       21.09       14.73      15.24   
BPS(원)       35342.00    37528.00    39406.00    42104.00   38053.00   
PBR(배)           1.09        1.49        2.06        1.93       1.25   
주당배당금(원)      1416.00     1416.00     2994.00     1692.00        NaN   
시가배당률(%)         3.66        2.54        3.70         NaN        NaN   
배당성향(%)         21.92       44.73       77.95         NaN        NaN   

                                                                  
             2020.06    2020.09    2020.12    2021.03 2021.06(E)  
주요재무정보                                                            
매출액        529661.00  669642.00  615515.00  653885.00  612813.00  
영업이익        81463.00  123532.00   90470.00   93829.00  109741.00  
당기순이익       55551.00   93607.00   66071.00   71417.00   84950.00  
영업이익률          15.38      18.45      14.70      14.35      17.91  
순이익률           10.49      13.98      10.73      10.92      13.86  
ROE(지배주주)       8.49       9.51       9.98      10.79        NaN  
부채비율           32.67      36.09      37.07      43.23        NaN  
당좌비율          250.04     229.69     214.82     192.26        NaN  
유보율         29477.97   30242.29   30692.79   30135.47        NaN  
EPS(원)        808.00    1364.00     949.00    1044.00    1205.00  
PER(배)         16.52      15.89      21.09      19.54      67.36  
BPS(원)      38534.00   39446.00   39406.00   39126.00        NaN  
PBR(배)          1.37       1.48       2.06       2.08        NaN  
주당배당금(원)         NaN        NaN        NaN        NaN        NaN  
시가배당률(%)         NaN        NaN        NaN        NaN        NaN  
배당성향(%)          NaN        NaN        NaN        NaN        NaN

In [29]:
# csv파일로 저장
financial_stmt.to_csv('Samsung_financial.csv', encoding='cp949', index=True)

# 웹크롤링 with Selenium

* 목표: 부산 호텔 추천 
* 사이트: 여기어때 https://www.goodchoice.kr/
* 수집항목: 이름/위치/평점

In [ ]:
# # cf. 다시 해보기 -> pc/moblie버전이 창 크기에 따라서 갈리는 것 같으니 처음부터 최대창으로 옵션 지정
# from selenium import webdriver

# # options: 브라우저를 최대한 크게 불러오기
# options = webdriver.ChromeOptions()
# options.add_argument("--start-maximized");

# # chrome browser 로드
# browser = webdriver.Chrome('chromedriver', options=options)

In [3]:
# Selenium을 이용한 크롤링
from selenium import webdriver

# 크롬 드라이버 불러오기
dr = webdriver.Chrome()

# 페이지 불러오기
dr.get('https://www.goodchoice.kr/')

# 검색 버튼 클릭하기: 검색창 띄우기
search = dr.find_element_by_css_selector('button.btn_srch')
search.click()

# 검색어 입력하기
keyword = dr.find_element_by_css_selector('div.wrap_inp input')
keyword.send_keys('부산')

# 엔터 누르기: 검색하기
from selenium.webdriver.common.keys import Keys
keyword.send_keys(Keys.RETURN)

In [4]:
# 스크랩한 호텔 수
total_hotels = 0

# 이름/가격/평점 정보를 담을 리스트 생성
names = []; prices = []; ratings = []

while total_hotels < 100:
    
    # 개별 호텔 전체 정보
    total_hotels += 1
    hotels = dr.find_elements_by_css_selector('div.stage')[:20]
    
    # 개별 호텔 이름/가격/평점 정보        
    for hotel in hotels:
        # 이름 저장
        name = hotel.find_element_by_css_selector('div.name strong').text
        names.append(name.lstrip().replace('\t','').replace('\n',''))
        # 가격 저장 
        price = hotel.find_element_by_css_selector('div.price').text.replace(",", "")
        prices.append(price.replace("\n",""))
        # 평점 저장
        rating = hotel.find_element_by_css_selector('p.score').text.replace(" ", "")
        ratings.append(rating.replace(" ",""))
                
#     print(names)
#     print(prices)    
#     print(ratings)

print('Crawling Finished!')

Crawling Finished!


In [5]:
# csv파일로 저장
f = open('Busan_hotel.csv', 'w')
f.write('이름, 가격, 평점\n')
for i in range(10):
    f.write(f'{names[i]}, , {prices[i]}, {rating[i]}\n')
f.close()

In [7]:
# 평점 크롤링이 잘못된듯 합니다... 
pd.read_csv('Busan_hotel.csv', encoding='cp949')

,이름,가격,평점
4성급해운대 센트럴 호텔,,남은 객실 1개 240000원,9
비지니스베스트웨스턴 플러스 부산송도,,132000원,.
비지니스해운대 썬클라우드 호텔,,숙박 다른날짜 확인,3
비지니스어반스테이 더 센트럴 서면,,남은 객실 1개 85000원,추
사직 덴바스타 시그니처호텔,,최대 12시간 무한대실예약특가 24500원12시부터 입실 숙박빠른입실예약특가 69...,천
비지니스[반짝특가] 라발스 호텔,,남은 객실 1개 195000원,해
2성급남포 오션투헤븐 호텔&스파,,남은 객실 1개 99000원,요
광안리 씨유,,최대 6시간 무한대실예약 30000원17시부터 입실 숙박빠른입실예약특가 80000원,(
레지던스[특가] 골든튤립 해운대 호텔&스위트,,200000145000원,7
3성급페어필드 바이 메리어트 부산 해운대,,99900원,0
